In [1]:
# import libraries
import pandas as pd
import numpy as np

# import data (using diabetic dataset)
diabetic_data = pd.read_csv('diabetic_data.csv')

#check if data is imported accurately
diabetic_data.head(5)

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [2]:
# Question 1: Melt format
# Used to transform the DataFrame from wide format -> long format.

# The operation will take 'encounter_id' as the identifier and melt 'metformin' and 'insulin' columns so that we have all the values under 'Medication' and their corresponding statuses under 'Med_Status'.
melted_diabetic_data = diabetic_data.melt(id_vars = ['encounter_id'], value_vars = ['metformin', 'insulin'], var_name = 'Medication', value_name = 'Med_Status')
print(melted_diabetic_data) 

# Rows before melting: 101766
# Rows after melting: 203532 

        encounter_id Medication Med_Status
0            2278392  metformin         No
1             149190  metformin         No
2              64410  metformin         No
3             500364  metformin         No
4              16680  metformin         No
...              ...        ...        ...
203527     443847548    insulin       Down
203528     443847782    insulin     Steady
203529     443854148    insulin       Down
203530     443857166    insulin         Up
203531     443867222    insulin         No

[203532 rows x 3 columns]


In [3]:
# Question 2: Pivot format

# Using pivot_table to transform the melted DataFrame back to a wide format
# aggfunc ='first' is used to get the first occurrence which is useful for categorical data.(to avoid repititions)

pivoted_diabetic_data = melted_diabetic_data.pivot_table(index='encounter_id', columns = 'Medication', values = 'Med_Status', aggfunc = 'first')

print(pivoted_diabetic_data)
# No of rows back to original no: 101766

Medication   insulin metformin
encounter_id                  
12522         Steady        No
15738         Steady        No
16680         Steady        No
28236         Steady        No
35754         Steady        No
...              ...       ...
443847548       Down    Steady
443847782     Steady        No
443854148       Down    Steady
443857166         Up        No
443867222         No        No

[101766 rows x 2 columns]


In [5]:
# Question 3: Aggregrate
# Using aggregate method to calculate mean, median, and standard deviation for num_lab_procedures?
aggregated_procedures = diabetic_data['num_lab_procedures'].aggregate(['mean', 'median', 'std'])

# Display the results
print(aggregated_procedures)

mean      43.095641
median    44.000000
std       19.674362
Name: num_lab_procedures, dtype: float64


In [6]:
# Question 4: Groupby
# Using Groupby calculate the mean 'number of lab procedures' based on 'race'?

# Race coulmn has missing data (?)
# First, replace missing data with "Unknown" values.
diabetic_data['race'] = diabetic_data['race'].replace('?', 'Unknown')

# Group by the 'race' and 'age' and calculate the mean for 'num_lab_procedures'
mean_lab_procedures = diabetic_data.groupby('race')['num_lab_procedures'].mean().reset_index()
print(mean_lab_procedures)

              race  num_lab_procedures
0  AfricanAmerican           44.085060
1            Asian           41.212168
2        Caucasian           42.832941
3         Hispanic           42.793324
4            Other           43.436919
5          Unknown           44.104707


In [7]:
# Combine .groupby() + .agg()

# Race coulmn has missing data (?)
# First, replace missing data with "Unknown" values.
diabetic_data['race'] = diabetic_data['race'].replace('?', 'Unknown')

# Groupby gender and aggregrate mean of 'time in hosp' and minimum and maximum 'num of medications' taken.
diabetic_data.groupby('gender').agg({
    'time_in_hospital': ['mean'],
    'num_medications': ['min', 'max']
})

time_in_hospital num_medications    
                            mean             min max
gender                                              
Female                  4.478797               1  75
Male                    4.299777               1  81
Unknown/Invalid         3.333333               6  22

In [9]:
#Question 5: Iteration

# Loop through each row in the diabetic_data DataFrame
for index, row in diabetic_data.iterrows():
    
# Check if the patient's race is Hispanic AND of age is between 10-20 years, are taking diabetes medication?
    if row['race'] == 'Hispanic'and row['age'] == '[10-20)' and (row['diabetesMed'] == 'Yes'):
        
# Print the details of the patient meeting the above conditions
        print(f"Patient number: {row['encounter_id']}, Race: {row ['race']}, Age: {row['age']}, Taking diabetes medication: {row['diabetesMed']}")

#This print the results of all the hispanic patients between age 10-20 who are taking diabetes medication

Patient number: 20572668, Race: Hispanic, Age: [10-20), Taking diabetes medication: Yes
Patient number: 37644138, Race: Hispanic, Age: [10-20), Taking diabetes medication: Yes
Patient number: 47101818, Race: Hispanic, Age: [10-20), Taking diabetes medication: Yes
Patient number: 47596488, Race: Hispanic, Age: [10-20), Taking diabetes medication: Yes
Patient number: 55600230, Race: Hispanic, Age: [10-20), Taking diabetes medication: Yes
Patient number: 55707072, Race: Hispanic, Age: [10-20), Taking diabetes medication: Yes
Patient number: 84684900, Race: Hispanic, Age: [10-20), Taking diabetes medication: Yes
Patient number: 106189350, Race: Hispanic, Age: [10-20), Taking diabetes medication: Yes
Patient number: 127377234, Race: Hispanic, Age: [10-20), Taking diabetes medication: Yes
Patient number: 172626198, Race: Hispanic, Age: [10-20), Taking diabetes medication: Yes
Patient number: 179674506, Race: Hispanic, Age: [10-20), Taking diabetes medication: Yes
Patient number: 187602960, R

In [11]:
# Another example: Iteration

# Define the columns we are interested to print
columns_to_print = ['encounter_id', 'race', 'age', 'diabetesMed', 'readmitted']

# Iterate through the DataFrame using the iterrows() function
for index, row in diabetic_data.iterrows():

# Check if the current index is less than 3, 
# which means we only want to print information from the first three rows
    if index < 3:
    
# Print the selected columns for the current row
        print(index,row[columns_to_print])
        print()
        
# If the index is 3 or higher, exit the loop using the break statement
    else:
        break

# This code iters through all the rows and prints only the first 3 rows (as mentioned) with the selected columns only.

0 encounter_id      2278392
race            Caucasian
age                [0-10)
diabetesMed            No
readmitted             NO
Name: 0, dtype: object

1 encounter_id       149190
race            Caucasian
age               [10-20)
diabetesMed           Yes
readmitted            >30
Name: 1, dtype: object

2 encounter_id              64410
race            AfricanAmerican
age                     [20-30)
diabetesMed                 Yes
readmitted                   NO
Name: 2, dtype: object

